# Weather Data Visualization Notebook

Welcome to the Weather Data Visualization Notebook! This notebook is designed to help you fetch and visualize weather data for the past N (N>0) days.

## Setup Instructions

Before you start using this notebook, please follow these steps to set up the Conda environment:

1. **Create and Activate the Conda Environment**:
   
   Ensure you have Conda installed. Open a terminal and navigate to the root directory of the repository. Run the following command to create a new Conda environment based on the `agro_eye_notebooks.yaml` file located in the `conda` folder:

   ```bash
   conda env create -f conda/agro_eye_notebooks.yaml
   ```
### Install env variable

Put the following env variables in your `~/.env`:
- "URL_OPEN-METEO"


In [1]:
# Import the necessary libraries
import requests
import json
from dotenv import load_dotenv
import os
import openmeteo_requests
import requests_cache
import pandas as pd
import numpy as np
from retry_requests import retry
import matplotlib as mpl
from matplotlib import pyplot as plt
import plotly.io as pio
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
load_dotenv(dotenv_path=os.path.expanduser("~/.env"))

True

In [3]:
default_env = {
    "URL_OPEN-METEO",
}

In [4]:
for env in default_env:
    if env not in os.environ:
        raise Exception(f"Missing environment variable: {env}. Please add it to your ~/.env file")

Fetch historical Weather Forcast. The information has been fecthed from [here](https://open-meteo.com/en/docs#hourly=&daily=weather_code,temperature_2m_max,temperature_2m_min,apparent_temperature_max,apparent_temperature_min,sunrise,sunset,daylight_duration,uv_index_max,precipitation_sum,rain_sum,snowfall_sum,precipitation_hours,wind_speed_10m_max,shortwave_radiation_sum,et0_fao_evapotranspiration&start_date=2024-07-01&end_date=2024-07-09&time_mode=time_interval)

In [5]:
# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

#### Query for daily information

In [6]:

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
params = {
	"latitude": 52.52,
	"longitude": 13.41,
	"daily": ["weather_code", "temperature_2m_max", "temperature_2m_min", "apparent_temperature_max", "apparent_temperature_min", "sunrise", "sunset", "daylight_duration", "uv_index_max", "precipitation_sum", "rain_sum", "snowfall_sum", "precipitation_hours", "wind_speed_10m_max", "shortwave_radiation_sum", "et0_fao_evapotranspiration"],
	"start_date": "2024-07-01",
	"end_date": "2024-07-09"
}
responses = openmeteo.weather_api(os.environ["URL_OPEN-METEO"], params=params)

In [7]:

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

Coordinates 52.52000045776367°N 13.419998168945312°E
Elevation 38.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


In [8]:
# Process daily data. The order of variables needs to be the same as requested.

daily = response.Daily()
daily_weather_code = daily.Variables(0).ValuesAsNumpy()
daily_temperature_2m_max = daily.Variables(1).ValuesAsNumpy()
daily_temperature_2m_min = daily.Variables(2).ValuesAsNumpy()
#daily_apparent_temperature_max = daily.Variables(3).ValuesAsNumpy()
#daily_apparent_temperature_min = daily.Variables(4).ValuesAsNumpy()
#daily_sunrise = daily.Variables(5).ValuesAsNumpy()
#daily_sunset = daily.Variables(6).ValuesAsNumpy()
daily_daylight_duration = daily.Variables(7).ValuesAsNumpy()
daily_uv_index_max = daily.Variables(8).ValuesAsNumpy()
daily_precipitation_sum = daily.Variables(9).ValuesAsNumpy()
daily_rain_sum = daily.Variables(10).ValuesAsNumpy()
daily_snowfall_sum = daily.Variables(11).ValuesAsNumpy()
daily_precipitation_hours = daily.Variables(12).ValuesAsNumpy()
daily_wind_speed_10m_max = daily.Variables(13).ValuesAsNumpy()
daily_shortwave_radiation_sum = daily.Variables(14).ValuesAsNumpy()
daily_et0_fao_evapotranspiration = daily.Variables(15).ValuesAsNumpy()


In [31]:
# Create a pandas DataFrame with the daily data
daily_data = {"date": pd.date_range(
	start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
	end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = daily.Interval()),
	inclusive = "left"
)}

#daily_data["weather_code"] = daily_weather_code
daily_data["temperature_2m_max"] = daily_temperature_2m_max
daily_data["temperature_2m_min"] = daily_temperature_2m_min
#daily_data["apparent_temperature_max"] = daily_apparent_temperature_max
#daily_data["apparent_temperature_min"] = daily_apparent_temperature_min
#daily_data["sunrise"] = daily_sunrise
#daily_data["sunset"] = daily_sunset
daily_data["daylight_duration"] = daily_daylight_duration
daily_data["uv_index_max"] = daily_uv_index_max
daily_data["precipitation_sum"] = daily_precipitation_sum
daily_data["rain_sum"] = daily_rain_sum
daily_data["snowfall_sum"] = daily_snowfall_sum
daily_data["precipitation_hours"] = daily_precipitation_hours
daily_data["wind_speed_10m_max"] = daily_wind_speed_10m_max
daily_data["shortwave_radiation_sum"] = daily_shortwave_radiation_sum
daily_data["et0_fao_evapotranspiration"] = daily_et0_fao_evapotranspiration


daily_data = pd.DataFrame(data = daily_data)

In [10]:
daily_data.head()

,date,temperature_2m_max,temperature_2m_min,daylight_duration,uv_index_max,precipitation_sum,rain_sum,snowfall_sum,precipitation_hours,wind_speed_10m_max,shortwave_radiation_sum,et0_fao_evapotranspiration
0,2024-07-01 00:00:00+00:00,19.995501,14.345500,60238.945312,5.45,5.0,4.5,0.0,6.0,15.077082,10.270000,2.063751
1,2024-07-02 00:00:00+00:00,17.295500,13.045500,60169.015625,3.95,1.4,1.4,0.0,6.0,16.171976,8.470000,1.728294
2,2024-07-03 00:00:00+00:00,17.345499,12.645500,60093.476562,5.35,0.0,0.0,0.0,0.0,12.287555,12.530000,2.457802
3,2024-07-04 00:00:00+00:00,18.895500,12.445499,60012.460938,4.35,2.8,2.7,0.0,6.0,20.380579,13.180000,2.446703
4,2024-07-05 00:00:00+00:00,21.395500,10.795500,59926.093750,5.75,0.0,0.0,0.0,0.0,17.906157,19.959999,4.121593


In [64]:
fig = go.Figure()

# Create a subplot layout: 4 rows, 1 column, shared x-axis
fig = make_subplots(rows=3, cols=1, shared_xaxes=True, vertical_spacing=0.01)

# Add the max temperature line with hover info
fig.add_trace(go.Scatter(x=daily_data['date'], y=daily_data['temperature_2m_max'].round(2),
                         mode='lines+markers',  # Added markers for better visibility on hover
                         name='Max Temperature',
                         line=dict(color='red', width=4),
                         hoverinfo='x+y',  # Shows both date and temperature on hover
                         hovertemplate='Date: %{x}<br>MaxTemperature: %{y}°C<extra></extra>',  # Custom hover text
                         showlegend=True  # Show the legend
                        ), row=1, col=1)

# Add the min temperature line with hover info
fig.add_trace(go.Scatter(x=daily_data['date'], y=daily_data['temperature_2m_min'].round(2),
                         mode='lines+markers',  # Added markers for better visibility on hover
                         name='Min Temperature',
                         line=dict(color='blue', width=4),
                         hoverinfo='x+y',  # Shows both date and temperature on hover
                         hovertemplate='Date: %{x}<br>Min Temperature: %{y}°C<extra></extra>',  # Custom hover text
                         showlegend=True  # Show the legend
                        ), row=1, col=1)


fig.add_trace(go.Scatter(x=daily_data['date'], y=daily_data['precipitation_sum'],
                         mode='lines+markers',
                         name='Precipitation Sum',
                         line=dict(color='brown', width=4),
                         hoverinfo='x+y',
                         hovertemplate='Date: %{x}<br>Precipitation: %{y}mm<extra></extra>'),
              row=2, col=1)

fig.add_trace(go.Scatter(x=daily_data['date'], y=daily_data['wind_speed_10m_max'].round(2),
                         mode='lines+markers',
                         name='Wind Speed Max (10m)',
                         line=dict(color='green', width=4),
                         hoverinfo='x+y',
                         hovertemplate='Date: %{x}<br>Wind Speed: %{y}Km/h<extra></extra>'),
              row=3, col=1)

# Update y-axes titles
fig.update_yaxes(tickfont=dict(size=12, weight='bold'), title_text="Temperature Min & Max (°C)", title_font=dict(size=18, weight='bold'), row=1, col=1)
fig.update_yaxes(tickfont=dict(size=12, weight='bold'), title_text="Precipitations Sum (mm)", title_font=dict(size=18, weight='bold'), row=2, col=1)
fig.update_yaxes(tickfont=dict(size=12, weight='bold'), title_text="Max Wind Speed (km/h)", title_font=dict(size=18, weight='bold'), row=3, col=1)
fig.update_xaxes(tickfont=dict(size=12, weight='bold'), row=3, col=1)

# Update layout (optional adjustments)
fig.update_layout(height=1000, width=1400, title_text="Weather Forecast Daily", title_font=dict(size=30, weight='bold'), title_x=0.5)

# Show the plot
fig.show()

In [65]:
#fig.write_image("../images/weather_forecast_daily.png")

#### Query for hourly information 

In [12]:
params = {
	"latitude": 52.52,
	"longitude": 13.41,
	"hourly": ["temperature_2m", "relative_humidity_2m", "precipitation", "rain", "snowfall", "surface_pressure", "evapotranspiration", "et0_fao_evapotranspiration", "wind_speed_10m", "soil_temperature_0cm", "soil_temperature_6cm", "soil_moisture_0_to_1cm", "soil_moisture_1_to_3cm", "uv_index"],
	"start_date": "2024-07-01",
	"end_date": "2024-07-09"
}
responses = openmeteo.weather_api(os.environ["URL_OPEN-METEO"], params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")


Coordinates 52.52000045776367°N 13.419998168945312°E
Elevation 38.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


In [13]:

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
hourly_relative_humidity_2m = hourly.Variables(1).ValuesAsNumpy()
hourly_precipitation = hourly.Variables(2).ValuesAsNumpy()
hourly_rain = hourly.Variables(3).ValuesAsNumpy()
hourly_snowfall = hourly.Variables(4).ValuesAsNumpy()
hourly_surface_pressure = hourly.Variables(5).ValuesAsNumpy()
hourly_evapotranspiration = hourly.Variables(6).ValuesAsNumpy()
hourly_et0_fao_evapotranspiration = hourly.Variables(7).ValuesAsNumpy()
hourly_wind_speed_10m = hourly.Variables(8).ValuesAsNumpy()
hourly_soil_temperature_0cm = hourly.Variables(9).ValuesAsNumpy()
hourly_soil_temperature_6cm = hourly.Variables(10).ValuesAsNumpy()
hourly_soil_moisture_0_to_1cm = hourly.Variables(11).ValuesAsNumpy()
hourly_soil_moisture_1_to_3cm = hourly.Variables(12).ValuesAsNumpy()
hourly_uv_index = hourly.Variables(13).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}

hourly_data["temperature_2m"] = hourly_temperature_2m
hourly_data["relative_humidity_2m"] = hourly_relative_humidity_2m
hourly_data["precipitation"] = hourly_precipitation
hourly_data["rain"] = hourly_rain
hourly_data["snowfall"] = hourly_snowfall
hourly_data["surface_pressure"] = hourly_surface_pressure
hourly_data["evapotranspiration"] = hourly_evapotranspiration
hourly_data["et0_fao_evapotranspiration"] = hourly_et0_fao_evapotranspiration
hourly_data["wind_speed_10m"] = hourly_wind_speed_10m
hourly_data["soil_temperature_0cm"] = hourly_soil_temperature_0cm
hourly_data["soil_temperature_6cm"] = hourly_soil_temperature_6cm
hourly_data["soil_moisture_0_to_1cm"] = hourly_soil_moisture_0_to_1cm
hourly_data["soil_moisture_1_to_3cm"] = hourly_soil_moisture_1_to_3cm
hourly_data["uv_index"] = hourly_uv_index

In [14]:
hourly_dataframe = pd.DataFrame(data = hourly_data)

In [61]:
fig = go.Figure()

# Create a subplot layout: 4 rows, 1 column, shared x-axis
fig = make_subplots(rows=4, cols=1, shared_xaxes=True, vertical_spacing=0.01)

# Add the temperature trace to the first subplot
fig.add_trace(go.Scatter(x=hourly_dataframe['date'], y=hourly_dataframe['temperature_2m'].round(2),
                         mode='lines+markers',
                         name='Temperature',
                         line=dict(color='red', width=2),
                         hoverinfo='x+y',
                         hovertemplate='Date: %{x}<br>Temperature: %{y}°C<extra></extra>'),
              row=1, col=1)

# Add the humidity trace to the second subplot
fig.add_trace(go.Scatter(x=hourly_dataframe['date'], y=hourly_dataframe['relative_humidity_2m'],
                         mode='lines+markers',
                         name='Humidity',
                         line=dict(color='brown', width=2),
                         hoverinfo='x+y',
                         hovertemplate='Date: %{x}<br>Humidity: %{y}%<extra></extra>'),
              row=2, col=1)

# Add the wind speed trace to the third subplot
fig.add_trace(go.Scatter(x=hourly_dataframe['date'], y=hourly_dataframe['wind_speed_10m'].round(2),
                         mode='lines+markers',
                         name='Wind Speed',
                         line=dict(color='green', width=2),
                         hoverinfo='x+y',
                         hovertemplate='Date: %{x}<br>Wind Speed:%{y} Km/h <extra></extra>'),
              row=3, col=1)

# Add the precipitation trace to the third subplot
fig.add_trace(go.Scatter(x=hourly_dataframe['date'], y=hourly_dataframe['precipitation'].round(2),
                         mode='lines+markers',
                         name='Precipitations',
                         line=dict(color='blue', width=2),
                         hoverinfo='x+y',
                         hovertemplate='Date: %{x}<br>Precipitation:%{y} mm <extra></extra>'),
              row=4, col=1)

#offset of the figure with respect to the point
fixed_offset = 3

# Iterate through the dataframe and add annotations
for index, row in hourly_dataframe.iterrows():
    hour = row['date'].hour
    # Determine if it's day (6 AM to 6 PM) or night
    if 6 <= hour <= 18:
        symbol = '☀️'  # Sun symbol for day
    else:
        symbol = '🌑'  # Moon symbol for night
    
    # Add annotation for each point
    fig.add_annotation(x=row['date'], y=round(row['temperature_2m'], 2) + 2,  # Adjust Y to place the symbol above the point
                       text=symbol,
                       showarrow=False,
                       font=dict(size=10),  # Adjust font size as needed
                       row=1, col=1)

# Update y-axes titles
fig.update_yaxes(title_text="Temperature (°C)", title_font=dict(weight='bold'), row=1, col=1)
fig.update_yaxes(title_text="Humidity (%)", title_font=dict(weight='bold'), row=2, col=1)
fig.update_yaxes(title_text="Wind Speed (Km/h)", title_font=dict(weight='bold'), row=3, col=1)
fig.update_yaxes(title_text="Precipitations (mm)", title_font=dict(weight='bold'), row=4, col=1)

# Update layout (optional adjustments)
fig.update_layout(height=800, width=1200, title_text="Weather Forecast Hourly", title_font=dict(size=30, weight='bold'), title_x=0.5)

# Show the plot
fig.show()

In [63]:
#fig.write_html("../images/weather_forecast_hourly.html")